# Account


In [2]:
# | default_exp classes.DomoAccount

In [3]:
# | exporti

from dataclasses import dataclass, field
from typing import Any, List

import datetime as dt
import re

import httpx

from nbdev.showdoc import patch_to

import domolibrary.utils.convert as cd
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.account as account_routes

import domolibrary.utils.chunk_execution as ce

In [4]:
# | export

from domolibrary.routes.account import (
    ShareAccount_V1_AccessLevel,
    ShareAccount_V2_AccessLevel,
    ShareAccount,
    GetAccount_NoMatch,
    ShareAccount_Error,
    ShareAccount_Error_AlreadyShared,
    DeleteAccount_Error,
)

from domolibrary.classes.DomoAccount_Config import (
    AccountConfig_UsesOauth,
    AccountConfig_ProviderTypeNotDefined,
    DomoAccount_Config,
    AccountConfig,
)


class Account_CanIModify(de.DomoError):
    def __init__(self, account_id, domo_instance):
        super().__init__(
            message=f"`DomoAccount.is_admin_summary` must be `False` to proceed.  Either set the value explicity, or retrieve the account instance using `DomoAccount.get_by_id()`",
            domo_instance=domo_instance,
        )


class UpsertAccount_MatchCriteria(de.DomoError):
    def __init__(self, domo_instance):
        super().__init__(
            message="must pass an account_id or account_name to UPSERT",
            domo_instance=domo_instance,
        )

# DomoAccount


In [5]:
# | export
@dataclass
class DomoAccount:
    id: int
    auth: dmda.DomoAuth = field(repr=False)

    name: str = None
    data_provider_type: str = None

    created_dt: dt.datetime = None
    modified_dt: dt.datetime = None

    config: DomoAccount_Config = None

    owner: List[Any] = None  # DomoUser or DomoGroup

    is_admin_summary: bool = True

    @classmethod
    def _from_json(
        cls,
        obj: dict,
        is_admin_summary: bool = True,
        auth: dmda.DomoAuth = None,
    ):
        """converts data_v1_accounts API response into an accounts class object"""

        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.id or dd.databaseId,
            name=dd.displayName,
            data_provider_type=dd.dataProviderId or dd.dataProviderType,
            created_dt=cd.convert_epoch_millisecond_to_datetime(
                dd.createdAt or dd.createDate
            ),
            modified_dt=cd.convert_epoch_millisecond_to_datetime(
                dd.modifiedAt or dd.lastModified
            ),
            auth=auth,
            is_admin_summary=is_admin_summary,
            owners = dd.owners
        )

## DomoAccount GET methods


In [6]:
# | export


class DomoAccounConfig_MissingFields(de.DomoError):
    def __init__(self, domo_instance, missing_keys, account_id):
        super().__init__(
            domo_instance=domo_instance,
            message=f"{account_id} config class definition is missing the following keys - {', '.join(keys)} extend the AccountConfig",
        )


@patch_to(DomoAccount)
def _test_missing_keys(self, res_obj, config_obj):
    return [r_key for r_key in res_obj.keys() if r_key not in config_obj.keys()]


@patch_to(DomoAccount)
async def _get_config(
    self: DomoAccount,
    session=None,
    return_raw: bool = False,
    debug_api: bool = None,
    auth: dmda.DomoAuth = None,
    debug_num_stacks_to_drop=2,
    is_suppress_no_config: bool = False,  # can be used to suppress cases where the config is not defined, either because the account_config is OAuth, and therefore not stored in Domo OR because the AccountConfig class doesn't cover the data_type
):
    if not self.data_provider_type:
        res = await account_routes.get_account_from_id(
            auth=self.auth,
            account_id=self.id,
            session=session,
            debug_api=debug_api,
            parent_class=self.__class__.__name__,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        )

        self.data_provider_type = res.response["dataProviderType"]

    res = await account_routes.get_account_config(
        auth=auth or self.auth,
        account_id=self.id,
        session=session,
        debug_api=debug_api,
        data_provider_type=self.data_provider_type,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    config_fn = AccountConfig(self.data_provider_type).value

    if not is_suppress_no_config and not config_fn.is_defined_config:
        raise config_fn._associated_exception(self.data_provider_type)

    self.config = config_fn._from_json(res.response)

    if self.config and self.config.to_json() != {}:
        if not res.response:
            print(self.data_provider_type, "no response")

        if not self.config.to_json():
            print(
                self.id,
                self.data_provider_type,
                "no config",
                self.config.to_json(),
                res.response,
            )

        self._test_missing_keys(res_obj=res.response, config_obj=self.config.to_json())

    return self.config

##### sample of hidden function \_get_config

called by `DomoAccount.get_by_id()`


In [7]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = DomoAccount(
    auth=token_auth,
    id=45,
)

try:
    res = await domo_account._get_config(
        debug_api=False, return_raw=False, is_suppress_no_config=False
    )
    print(res)

except AccountConfig_UsesOauth as e:
    print(e)

data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


In [8]:
# | export
@patch_to(DomoAccount, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    account_id: int,
    is_suppress_no_config: bool = True,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    """retrieves account metadata and attempts to retrieve config"""

    res = await account_routes.get_account_from_id(
        auth=auth,
        account_id=account_id,
        session=session,
        debug_api=debug_api,
        parent_class=cls.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    acc = cls._from_json(obj=res.response, auth=auth, is_admin_summary=False)

    await acc._get_config(
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        is_suppress_no_config=is_suppress_no_config,
    )

    return acc

#### sample implementation of get_by_id

this implementation returns a result because accoun_id 5 is linked to a data_provider (domo_governance) that has been defined in the `DomoAccount_Config.AccountConfig` enum


In [29]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoAccount.get_by_id(
    auth=token_auth,
    account_id=71,
    return_raw=False,
    debug_api=False,
    is_suppress_no_config=True,
)

DomoAccount(id=71, name='DomoLibrary - testrename 2024-02-21', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 2, 21, 0, 48, 6), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

This implementation fails because the data_provider_type has not been defined in `DomoAccount_Config.AccountConfig`


In [10]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


await DomoAccount.get_by_id(
    auth=token_auth, account_id=45, return_raw=False, debug_api=False
)

data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


DomoAccount(id=45, name='test-goolesheets', data_provider_type='google-spreadsheets', created_dt=datetime.datetime(2022, 10, 4, 22, 59, 14), modified_dt=datetime.datetime(2023, 4, 24, 15, 40, 28), config=DomoAccount_NoConfig_OAuth(is_oauth=True, is_defined_config=False), owner=None, is_admin_summary=False)

## DomoAccount CRUD Methods


### Create Account


In [11]:
# | export


@patch_to(DomoAccount, cls_method=True)
async def create_account(
    cls: DomoAccount,
    account_name: str,
    config: DomoAccount_Config,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    body = account_routes.generate_create_body(account_name=account_name, config=config)

    res = await account_routes.create_account(
        auth=auth, config_body=body, debug_api=debug_api, session=session
    )

    if return_raw:
        return res

    return await cls.get_by_id(auth=auth, account_id=res.response.get("id"))

### Update DomoAccount


In [12]:
# | export


@patch_to(DomoAccount)
async def update_config(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    config: DomoAccount_Config = None,
    is_suppress_no_config=False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    auth = auth or self.auth
    config = config or self.config

    res = await account_routes.update_account_config(
        auth=auth,
        account_id=self.id,
        config_body=config.to_json(),
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success and self.is_admin_summary:
        raise Account_CanIModify(account_id=self.id, domo_instance=auth.domo_instance)

    await self._get_config(
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        is_suppress_no_config=is_suppress_no_config,
    )

    return self

#### Sample implementation of update_config

to update the account config,

- assign `DomoAccount` class instance `domo_account.config` a `DomoAccount` class.<br>
  - the `AccountConfig` enumerates all of the `DomoAccount`s currently available. _issue a pull request to add a new one_
  - each member of `AccountConfig` uses the camel_case spelling of the data-provider-type


In [13]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=71)


# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.config.domo_instance = "any_new_value_goes_here"
await domo_account.update_config()

DomoAccount(id=71, name='domo_creds - update 2024-02-21', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 2, 21, 0, 20, 58), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

The config can be updated by passing a new `AccountConfig` object


In [14]:
import os
import json

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=71)

# update domo Account API by passing new config object
config = AccountConfig.abstract_credential_store.value(
    credentials=json.dumps({"domo_instance": token_auth.domo_instance})
)
await domo_account.update_config(config=config)

DomoAccount(id=71, name='domo_creds - update 2024-02-21', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 2, 21, 0, 20, 58), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

In [15]:
# | export
@patch_to(DomoAccount)
async def update_name(
    self: DomoAccount,
    account_name: str = None,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    auth = auth or self.auth

    res = await account_routes.update_account_name(
        auth=auth,
        account_id=self.id,
        account_name=account_name or self.name,
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success and self.is_admin_summary:
        raise Account_CanIModify(account_id=self.id, domo_instance=auth.domo_instance)

    await self.get_by_id(auth=auth, account_id=self.id)

    return self

#### sample implementation of update_name

`DomoAccount` can be updated by altering properties and then calling `DomoAccount.update_name()` or by passing the account_name parameter


In [16]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=71)
domo_account.name = f"DomoLibrary - update {dt.datetime.now()}"
await domo_account.update_name()

DomoAccount(id=71, name='DomoLibrary - update 2024-02-21 00:48:04.579046', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 2, 21, 0, 20, 58), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

In [17]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=71)

await domo_account.update_name(
    account_name=f"DomoLibrary - testrename {dt.date.today()}", return_raw=False
)

DomoAccount(id=71, name='DomoLibrary - update 2024-02-21 00:48:04.579046', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 2, 21, 0, 48, 4), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

### Delete Account


In [18]:
# | export


@patch_to(DomoAccount)
async def delete_account(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=2,
    parent_class=None,
):
    auth = auth or self.auth

    res = await account_routes.delete_account(
        auth=auth,
        account_id=self.id,
        debug_api=debug_api,
        session=session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success and self.is_admin_summary:
        raise Account_CanIModify(account_id=self.id, domo_instance=auth.domo_instance)

    return res

### Share DomoAccount


In [19]:
# | export
@patch_to(DomoAccount)
async def is_feature_accountsv2_enabled(
    self: DomoAccount, auth: dmda.DomoFullAuth = None, return_raw: bool = False
):
    """uses bootstrap class to test if the auth object refers to an instancce that has the account-v2 feature switch enabled"""
    import domolibrary.classes.DomoBootstrap as dmbs

    auth = auth or self.auth

    domo_bsr = dmbs.DomoBootstrap(auth=auth)

    try:
        is_v2 = await domo_bsr.is_feature_accountsv2_enabled()
        return 1 if is_v2 else 0

    except dmbs.InvalidAuthTypeError as e:
        print(
            f"Warning - unable to test if accounts_v2 feature is enabled in {auth.domo_instance}, recommend pass FullAuth"
        )
        return -1

#### sample implementation of is_veature_accountsv2_enabled

In [20]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

community_account = await DomoAccount.get_by_id(account_id=71, auth=full_auth)
assert community_account

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-alpha",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

alpha_account = await DomoAccount.get_by_id(account_id=5, auth=full_auth)
assert alpha_account

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-alpha",
    domo_access_token=os.environ["ALPHA_ACCESS_TOKEN"],
)
alpha_account_token = await DomoAccount.get_by_id(account_id=5, auth=token_auth)
assert alpha_account_token

{
    "domo-community-full_auth": await community_account.is_feature_accountsv2_enabled(
        return_raw=False
    ),
    "domo-alpha-full_auth": await alpha_account.is_feature_accountsv2_enabled(
        return_raw=False
    ),
    "domo-alpha-token_auth": await alpha_account_token.is_feature_accountsv2_enabled(
        return_raw=False
    ),
}

Warning - unable to test if accounts_v2 feature is enabled in domo-alpha, recommend pass FullAuth


{'domo-community-full_auth': 0,
 'domo-alpha-full_auth': 1,
 'domo-alpha-token_auth': -1}

In [21]:
# | export
@patch_to(DomoAccount)
async def _share_v2(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    user_id=None,
    group_id=None,
    access_level: ShareAccount = ShareAccount_V2_AccessLevel.CAN_VIEW,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
    is_suppress_already_shared: bool = True,
):
    auth = auth or self.auth

    is_v2 = await self.is_feature_accountsv2_enabled(auth=auth)

    if is_v2 == 0:
        raise account_routes.ShareAccount_Error(
            account_id=self.id,
            response="accounts_v2 feature not enabled, use v1 share method",
            domo_instance=auth.domo_instance,
            function_name="_share_v2",
            parent_class=self.__class__.__name__,
            status=None,
        )

    share_payload = account_routes.generate_share_account_payload_v2(
        user_id=user_id,
        group_id=group_id,
        access_level=access_level,
    )
    try:
        res = await account_routes.share_account_v2(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
            parent_class=self.__class__.__name__,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        )

        if return_raw:
            return res

        return f"shared {self.id} - {self.name} with {group_id or user_id}"

    except ShareAccount_Error_AlreadyShared as e:
        if not is_suppress_already_shared:
            raise e

        return f"already shared {self.id} - {self.name} with {group_id or user_id}"

#### sample implementation of share_account_v2


In [22]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-alpha",
    domo_access_token=os.environ["ALPHA_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(account_id=5, auth=token_auth)
try:
    print(
        await domo_account._share_v2(
            access_level=ShareAccount_V2_AccessLevel.CAN_VIEW,
            group_id=1814479647,
            auth=token_auth,
            debug_api=False,
            is_suppress_already_shared=True,
        )
    )

except ShareAccount_Error as e:
    print(e)

Warning - unable to test if accounts_v2 feature is enabled in domo-alpha, recommend pass FullAuth
already shared 5 - Abstract Credential Store Account with 1814479647


In [23]:
# | export


@patch_to(DomoAccount)
async def _share_v1(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    user_id=None,
    access_level: ShareAccount = ShareAccount_V1_AccessLevel.CAN_VIEW,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
    is_suppress_already_shared: bool = True,
):
    auth = auth or self.auth

    is_v2 = await self.is_feature_accountsv2_enabled(auth=auth)

    if is_v2 == 1:
        raise account_routes.ShareAccount_Error(
            account_id=self.id,
            response="accounts_v2 feature enabled, use v2 share method",
            domo_instance=auth.domo_instance,
            function_name="_share_v2",
            parent_class=self.__class__.__name__,
            status=None,
        )

    share_payload = account_routes.generate_share_account_payload_v1(
        user_id=user_id,
        access_level=access_level,
    )
    try:
        res = await account_routes.share_account_v1(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
            parent_class=self.__class__.__name__,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        )

        if return_raw:
            return res

        return f"shared {self.id} - {self.name} with { user_id}"

    except ShareAccount_Error_AlreadyShared as e:
        if is_suppress_already_shared:
            return f"already shared {self.id} - {self.name} with { user_id}"

        else:
            raise e

#### sample implementation of share_account_v1


In [24]:
import os
import domolibrary.classes.DomoGroup as dmdg
import domolibrary.utils.chunk_execution as ce

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(account_id=45, auth=token_auth)
domo_group = await dmdg.DomoGroup.get_by_id(group_id=1814479647, auth=token_auth)

domo_users = await domo_group.Membership.get_members()
domo_user_ids = [domo_user.id for domo_user in domo_users]

try:
    print(
        await ce.gather_with_concurrency(
            *[
                domo_account._share_v1(
                    access_level=ShareAccount_V1_AccessLevel.CAN_EDIT,
                    user_id=user_id,
                    auth=token_auth,
                    debug_api=False,
                )
                for user_id in domo_user_ids
            ],
            n=10,
        )
    )

except ShareAccount_Error as e:
    print(e)

data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
Warning - unable to test if accounts_v2 feature is enabled in domo-community, recommend pass FullAuth
Warning - unable to test if accounts_v2 feature is enabled in domo-community, recommend pass FullAuth
['shared 45 - test-goolesheets with 55874022', 'already shared 45 - test-goolesheets with 1893952720']


In [25]:
# | exporti


@patch_to(DomoAccount)
async def share(
    self: DomoAccount,
    user_id=None,
    group_id=None,
    domo_user=None,
    domo_group=None,
    auth: dmda.DomoAuth = None,
    access_level: ShareAccount = None,  # will default to Read
    is_suppress_already_shared: bool = True,
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 3,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth
    is_v2 = await self.is_feature_accountsv2_enabled(auth=auth)

    user_id = user_id or (domo_user and domo_user.id)

    debug = {"is_accounts_v2": is_v2}

    res = None

    if is_v2 == 1:
        group_id = group_id or (domo_group and domo_group.id)

        debug.update(
            {
                "user_id*": user_id,
                "group_id": group_id,
            }
        )

        if debug_prn:
            print(debug)

        res = await self._share_v2(
            auth=auth,
            user_id=user_id,
            group_id=group_id,
            debug_api=debug_api,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            session=session,
            is_suppress_already_shared=is_suppress_already_shared,
        )

    elif is_v2 == 0:
        user_ids = [user_id]

        if group_id:
            import domolibrary.classes.DomoGroup as dmdg

            domo_group = await dmdg.DomoGroup.get_by_id(group_id=group_id, auth=auth)

        group_id = group_id or domo_group.id
        domo_users = await domo_group.Membership.get_members()
        user_ids = [domo_user.id for domo_user in domo_users]

        debug.update({"group_id": group_id, "user_ids": user_ids})

        if debug_prn:
            print(debug)

        res = await ce.gather_with_concurrency(
            *[
                self._share_v1(
                    auth=auth,
                    user_id=user_id,
                    debug_api=debug_api,
                    debug_num_stacks_to_drop=debug_num_stacks_to_drop,
                    session=session,
                    is_suppress_already_shared=is_suppress_already_shared,
                )
                for user_id in user_ids
            ],
            n=10,
        )

    return res

#### sample implementation of domo_account.share


In [26]:
import os
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoGroup as dmg

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)
account_id = 71
domo_account = await DomoAccount.get_by_id(account_id=account_id, auth=full_auth)
assert domo_account

domo_group = await dmg.DomoGroup.get_by_id(group_id=1814479647, auth=token_auth)
assert domo_group

domo_users = await domo_group.Membership.get_members()
assert domo_users

await domo_account.share(
    domo_group=domo_group,
    access_level=ShareAccount_V1_AccessLevel.CAN_VIEW,
    auth=full_auth,
    debug_prn=True,
    debug_api=False,
)

CancelledError: 

# DomoAccounts

A class for retrieving account objects


In [ ]:
# | export


@dataclass
class DomoAccounts:
    auth: dmda.DomoAuth

In [ ]:
# | export


@staticmethod
@patch_to(DomoAccounts)
async def _get_accounts_accountsapi(
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    res = await account_routes.get_accounts(
        auth=auth, debug_api=debug_api, session=session
    )

    if return_raw:
        return res

    if len(res.response) == 0:
        return []

    return await ce.gather_with_concurrency(
        n=60,
        *[
            DomoAccount.get_by_id(
                account_id=json_obj.get("id"),
                debug_api=debug_api,
                session=session,
                auth=auth,
            )
            for json_obj in res.response
        ],
    )


@patch_to(DomoAccounts, cls_method=True)
async def _get_accounts_queryapi(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    additional_filters_ls=None,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    """v2 api for works with group_account_v2 beta"""

    import domolibrary.routes.datacenter as datacenter_routes

    res = await datacenter_routes.search_datacenter(
        auth=auth,
        entity_type=datacenter_routes.Datacenter_Enum.ACCOUNT.value,
        additional_filters_ls=additional_filters_ls,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    if len(res.response) == 0:
        return []

    return [
        DomoAccount._from_json(account_obj, auth=auth) for account_obj in res.response
    ]


@patch_to(DomoAccounts, cls_method=True)
async def get_accounts(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    additional_filters_ls=None,  # datacenter_routes.generate_search_datacenter_filter
    # account string to search for, must be an exact match in spelling.  case insensitive
    # v2 will use the queryAPI as it returns more complete results than the accountsAPI
    is_v2: bool = None,
    is_suppress_undefined_provider_type: bool = False,
    account_name: str = None,
    account_id: str = None,
    account_type: AccountConfig = None,  # to retrieve a specific account type
    account_type_str=None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_prn: bool = False,
):
    import domolibrary.classes.DomoBootstrap as bsr
    import domolibrary.routes.datacenter as datacenter_routes

    if isinstance(auth, dmda.DomoFullAuth) and is_v2 is None:
        instance_bsr = bsr.DomoBootstrap(auth=auth)

        is_v2 = await instance_bsr.is_feature_accountsv2_enabled(auth)

        if debug_prn:
            print(
                f"{auth.domo_instance} {'is' if is_v2 else 'is not'} using the v2 beta"
            )

    if is_v2:
        try:
            domo_accounts = await cls._get_accounts_queryapi(
                auth=auth,
                debug_api=debug_api,
                additional_filters_ls=additional_filters_ls,
                session=session,
            )
        except datacenter_routes.SearchDatacenter_NoResultsFound as e:
            print(e)
            domo_accounts = []
    else:
        domo_accounts = await cls._get_accounts_accountsapi(
            auth=auth, debug_api=debug_api, session=session
        )

    if return_raw or len(domo_accounts) == 0:
        return domo_accounts

    if account_id:
        domo_account = next(
            (
                domo_account
                for domo_account in domo_accounts
                if int(domo_account.id) == int(account_id)
            ),
            None,
        )

        if not domo_account:
            raise GetAccount_NoMatch(
                account_id=account_id, domo_instance=auth.domo_instance
            )

        return domo_account

    if account_name and isinstance(account_name, str):
        domo_accounts = [
            domo_account
            for domo_account in domo_accounts
            if domo_account.name.lower() == account_name.lower()
        ]

    if account_type:
        return [
            domo_account
            for domo_account in domo_accounts
            if domo_account.data_provider_type == account_type.value.data_provider_type
        ]

    if account_type_str:
        return [
            domo_account
            for domo_account in domo_accounts
            if domo_account.data_provider_type == account_type_str
        ]

    return domo_accounts

#### sample get_accounts


In [ ]:
import os

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoAccounts.get_accounts(
    auth=auth,
    is_v2=False,
    # account_name = 'domo_creds',
    account_id=71,
    return_raw=False,
    debug_prn=False,
    debug_api=False,
)

data provider type domo-csv not defined yet. Extend the AccountConfig class
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


DomoAccount(id=71, name='DomoLibrary - testrename 2024-01-05', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 1, 5, 18, 14, 35), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

In [ ]:
# |export
@dataclass
class Account_Accesslist_Share:
    entity: Any
    access_level: ShareAccount
    auth: dmda.DomoAuth

    @staticmethod
    async def _get_entity(obj, auth: dmda.DomoAuth):
        if obj["type"] == "USER":
            import domolibrary.classes.DomoUser as dmu

            return await dmu.DomoUser.get_by_id(user_id=obj["id"], auth=auth)

        if obj["type"] == "GROUP":
            import domolibrary.classes.DomoGroup as dmg

            return await dmg.DomoGroup.get_by_id(group_id=obj["id"], auth=auth)

        return None

    @staticmethod
    def _get_access_level(access_level, is_v2: int):
        if is_v2 == 1:
            return ShareAccount_V2_AccessLevel[access_level]

        else:
            return ShareAccount_V1_AccessLevel[access_level]

    @classmethod
    async def _from_json(
        cls: ShareAccount, obj, auth: dmda.DomoAuth, is_v2: bool = False
    ):
        return cls(
            entity=await cls._get_entity(obj, auth=auth),
            auth=auth,
            access_level=cls._get_access_level(obj["accessLevel"], is_v2),
        )


@dataclass
class Account_Accesslist:
    account: DomoAccount
    auth: dmda.DomoAuth
    domo_users = None
    domo_groups = None

In [ ]:
# | exporti
@patch_to(DomoAccount)
async def get_accesslist(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth
    res = await account_routes.get_account_accesslist(
        auth=auth, account_id=self.id, debug_api=debug_api, session=session
    )

    if return_raw:
        return res

    is_v2 = await self.is_feature_accountsv2_enabled()

    self.accesslist = await ce.gather_with_concurrency(
        *[
            Account_Accesslist_Share._from_json(obj=obj, auth=auth, is_v2=is_v2)
            for obj in res.response["list"]
        ],
        n=10,
    )
    return self.accesslist

#### sample implementation of get_accesslist


In [ ]:
import os

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=auth, account_id=71)

await domo_account.get_accesslist()

Warning - unable to test if accounts_v2 feature is enabled in domo-community, recommend pass FullAuth


[Account_Accesslist_Share(entity=DomoUser(id='55874022', display_name='Grant Smith', email_address='grantsmith@gmail.com', role_id=1, department='', title='', avatar_key='932c359c-5e94-40a4-8f0f-bb67e61b034d', phone_number='0', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=1618632000000, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=1, name='Admin', description='Full access to everything', is_system_role=True, is_default_role=None, grant_ls=[], membership_ls=[])), access_level=<ShareAccount_V1_AccessLevel.CAN_VIEW: 'READ'>, auth=DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true')),
 Account_Accesslist_Share(entity=DomoUser(id='1893952720', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, department='Bu

# UPSERT Functions


In [ ]:
# | exporti


@patch_to(DomoAccounts, cls_method=True)
async def upsert_account(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    account_config: AccountConfig = None,
    account_name: str = None,
    account_id: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    """search for an account and upsert it"""

    if not account_name and not account_id:
        raise UpsertAccount_MatchCriteria(domo_instance=auth.domo_instance)

    acc = None
    res = None

    if account_id:
        acc = await DomoAccounts.get_accounts(account_id=account_id, auth=auth)

        if acc and account_name:
            if debug_prn:
                print(f"upsertting {acc.id}:  updating account_name")
            res = await acc.update_name(
                account_name=account_name, debug_api=debug_api, return_raw=return_raw
            )

    if account_name and acc is None:
        acc = await DomoAccounts.get_accounts(
            account_name=account_name,
            auth=auth,
            account_type_str=(account_config and account_config.data_provider_type)
            or None,
            # is_suppress_undefined_provider_type = True
        )

        if isinstance(acc, list) and len(acc) > 0 and isinstance(acc[0], DomoAccount):
            acc = acc[0]

        else:
            acc = None

    if acc and account_config:  # upsert account
        acc.config = account_config

        if debug_prn:
            print(f"upsertting {acc.id}:  updating config")

        res = await acc.update_config(debug_api=debug_api, return_raw=return_raw)

    if return_raw and acc:
        return res

    if not acc:
        if debug_prn:
            print(f"creating account {account_name} in {auth.domo_instance}")

        acc = await DomoAccount.create_account(
            account_name=account_name,
            config=account_config,
            auth=auth,
            debug_api=debug_api,
            return_raw=return_raw,
        )

    return acc

#### sample implementation of upsert_account


In [ ]:
import os
import domolibrary.classes.DomoUser as dmu
import datetime as dt

dt.date.today()

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

account_config = AccountConfig.abstract_credential_store.value(
    credentials=f"abc {dt.date.today()}"
)


# creates a DomoAccount object
await DomoAccounts.upsert_account(
    account_name=f"domo_creds - update {dt.date.today()}",
    account_id=71,
    auth=token_auth,
    debug_api=False,
    debug_prn=True,
    account_config=account_config,
    return_raw=False,
)

data provider type domo-csv not defined yet. Extend the AccountConfig class
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
upsertting 71:  updating account_name
upsertting 71:  updating config


DomoAccount(id=71, name='DomoLibrary - testrename 2024-01-05', data_provider_type='abstract-credential-store', created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2024, 1, 5, 18, 14, 35), config=DomoAccount_Config_AbstractCredential(credentials='********'), owner=None, is_admin_summary=False)

In [ ]:
# | exporti


@patch_to(DomoAccount)
async def upsert_share_account_user(
    self: DomoAccount,
    domo_user,
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    ls_share = await account_routes.get_account_accesslist(
        auth=auth, account_id=self.id
    )
    res = None

    if domo_user:
        user_id = domo_user.id
        found_user = next(
            (
                obj
                for obj in ls_share.response["list"]
                if obj["id"] == user_id and obj["type"] == "USER"
            ),
            None,
        )
        if not found_user:
            res = await self.share(
                domo_user=domo_user,
                auth=auth,
                access_level=access_level,
                debug_api=debug_api,
                debug_prn=debug_prn,
                session=session,
            )

    return res


@patch_to(DomoAccount)
async def upsert_share_account_group(
    self: DomoAccount,
    domo_group,
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    ls_share = await account_routes.get_account_accesslist(
        auth=auth, account_id=self.id
    )
    res = None

    if domo_group:
        group_id = domo_group.id
        found_group = next(
            (
                obj
                for obj in ls_share.response["list"]
                if obj["id"] == group_id and obj["type"] == "GROUP"
            ),
            None,
        )
        if not found_group:
            res = await self.share(
                domo_group=domo_group,
                auth=auth,
                access_level=access_level,
                debug_api=debug_api,
                debug_prn=debug_prn,
                session=session,
            )

    return res

#### sample implementation upsert share account


In [ ]:
import os
import domolibrary.classes.DomoUser as dmu

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=auth, account_id=71)
domo_user = await dmu.DomoUser.get_by_id(user_id=68216396, auth=auth)

await domo_account.upsert_share_account_user(domo_user=domo_user)

Warning - unable to test if accounts_v2 feature is enabled in domo-community, recommend pass FullAuth


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()